In [1]:
import os
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit

pd.set_option('display.max_Columns', 100)

In [2]:
No = 1

In [3]:
INPUT_DIR = '../input/'
OUTPUT_DIR = '../output/'

In [4]:
def read_feather(filename):
    df = pd.read_feather(os.path.join(INPUT_DIR, filename))
    print(filename, df.shape)
    return df

In [5]:
train_df = read_feather('train.f')
test_df = read_feather('test.f')

train.f (1997595, 35)
test.f (390095, 30)


In [6]:
campaign_df = read_feather('campaign.f')
map_gv_df = read_feather('map_game_feed_native_video_assets.f')
ad_video_df = read_feather('advertiser_video.f')
ad_cvideo_df = read_feather('advertiser_converted_video.f')

campaign.f (14627, 4)
map_game_feed_native_video_assets.f (2796, 3)
advertiser_video.f (11707, 6)
advertiser_converted_video.f (198622, 8)


In [7]:
ad_cvideo_df = ad_cvideo_df.drop_duplicates(
    subset=['mst_advertiser_video_id', 
                   'mst_game_feed_id', 
                    'mst_video_template_id'], keep='last')

le = preprocessing.LabelEncoder()
le.fit(['vertical', 'horizontal'])
ad_cvideo_df['rectangle_type_id'] = le.transform(ad_cvideo_df['rectangle_type'])
ad_cvideo_df.drop(columns=['rectangle_type'], inplace=True)
print('ad_cvideo_df', ad_cvideo_df.shape)

ad_cvideo_df (107493, 8)


### map_game_feed_native_video_assetsをマージ

In [8]:
def merge_gamefeed_features(input_df):
    input_merge = pd.merge(input_df, map_gv_df, 
                           left_on='game_feed_id', right_on='mst_game_feed_id', how='left').drop(columns=['mst_game_feed_id'])
    
    horizontal = ad_video_df.copy()
    left_keys = ['horizontal_mst_advertiser_video_id', 'advertiser_id']
    right_keys = ['id', 'mst_advertiser_id']
    horizontal.columns = [f'horizontal_{c}' if c not in right_keys else c for c in horizontal.columns]
    input_merge = pd.merge(input_merge, horizontal, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys) 
    
    vertical = ad_video_df.copy()
    left_keys = ['vertical_mst_advertiser_video_id', 'advertiser_id']
    right_keys = ['id', 'mst_advertiser_id']
    vertical.columns = [f'vertical_{c}' if c not in right_keys else c for c in vertical.columns]
    input_merge = pd.merge(input_merge, vertical, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys)
    
    left_keys = [
        "horizontal_mst_advertiser_video_id",
        "game_feed_id",
        "video_template_id",
    ]
    right_keys = [
        "mst_advertiser_video_id",
        "mst_game_feed_id",
        "mst_video_template_id",
    ]
    horizontal = ad_cvideo_df.copy()
    horizontal.columns = [f"horizontal_converted_{c}" if c not in right_keys else c for c in horizontal.columns]
    input_merge = pd.merge(input_merge, horizontal, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys) 
    
    left_keys = [
        "vertical_mst_advertiser_video_id",
        "game_feed_id",
        "video_template_id",
    ]
    right_keys = [
        "mst_advertiser_video_id",
        "mst_game_feed_id",
        "mst_video_template_id",
    ]
    vertical = ad_cvideo_df.copy()
    vertical.columns = [f"vertical_converted_{c}" if c not in right_keys else c for c in vertical.columns]
    input_merge = pd.merge(input_merge, vertical, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys)
    
#     input_merge.drop(columns=['game_feed_id', 'advertiser_id', 'video_template_id', 
#                               'horizontal_mst_advertiser_video_id', 'vertical_mst_advertiser_video_id'], inplace=True)
    
    # merge
    merge_col = ['duration', 'file_size', 'converted_file_size', 'converted_bitrate']
    vert_horz = ['vertical_', 'horizontal_']
    
    for m_col in merge_col:
        input_merge[m_col] = 0
        for vh in vert_horz:
            input_merge[m_col] = input_merge[m_col] + input_merge[vh+m_col].fillna(0)
            input_merge.drop(columns=[vh+m_col], inplace=True)
    
    return input_merge

### campaignをマージ

In [9]:
def merge_campaign_features(input_df):
    campaign = pd.merge(input_df, campaign_df, left_on='campaign_id', right_on='id', how='left')
    campaign.drop(columns=['id'], inplace=True)
    return campaign

In [10]:
def merge_all(input_df):
    input_df = merge_gamefeed_features(input_df)
    input_df = merge_campaign_features(input_df)
    print('merge', input_df.shape)
    return input_df

In [11]:
print('train_df', train_df.shape)
train_df = merge_all(train_df)
print('test_df', test_df.shape)
test_df = merge_all(test_df)

train_df (1997595, 35)
merge (1997595, 54)
test_df (390095, 30)
merge (390095, 49)


In [12]:
whole_df = pd.concat([train_df, test_df], ignore_index=True)
print('whole_df', whole_df.shape)

whole_df (2387690, 54)


In [13]:
label_encoding_col = [
    'adspot_id',
    'adspot_video_format_id', 
    'country_code', 
    'game_feed_asset_type_id',
    'item_id', 
    'os', 
    'video_template_id',
    'uid',
#     'auction_type_id', ->OHE
#     'user_type_id',  ->OHE
    
#     'advertiser_id',  ->TGE
#     'app_id',   ->TGE
#     'media_app_id',   ->TGE
#     'campaign_id',  ->TGE
    'category_id', 
    'game_feed_id', 
    'game_template_id',
    
    'horizontal_mst_advertiser_video_id',
    'vertical_mst_advertiser_video_id', 
    'horizontal_converted_rectangle_type_id', 
    'vertical_converted_rectangle_type_id',
    'mst_advertiser_id', 
    'mst_advertiser_order_id', 
    'mst_user_type_id',
]

In [14]:
target_enc_col = [
#     'advertiser_id', 
    'app_id', 
    'media_app_id', 
    'campaign_id',
#     'category_id',
#     'game_feed_id',
#     'game_template_id',
]

In [15]:
count_enc_col = [
    'advertiser_id', 
    'adnw_id', 
    'adspot_id',
    'category_id', 
    'game_feed_id',
    'uid', 
    'game_template_id',
    
    'mst_advertiser_order_id', 
    'mst_user_type_id',
]

In [16]:
onehot_enc_col = [
    'auction_type_id',
    'header_bidding',
    'is_interstitial',
    'user_type_id'
]

In [17]:
continuous_col = [
    'first_login_interval',
    'max_login_interval', 
    'frequency', 
    'login_frequency', 
    'last_login_interval',
    'from_click',
    'pos',
#     'adnw_id',
#     'header_bidding', 
#     'is_interstitial'
    
    'horizontal_width',
    'horizontal_height', 
    'vertical_width', 
    'vertical_height',
    'horizontal_converted_width', 
    'horizontal_converted_height',
    'vertical_converted_width',
    'vertical_converted_height', 
    'duration', 
    'file_size', 
    'converted_file_size', 
    'converted_bitrate',
]

### Label Encoding

In [18]:
def get_non_overlapping(column: str):
    """train/testにしか出てこない値を調べる"""
    only_in_train = set(train_df[column].unique()) - set(test_df[column].unique())
    only_in_test = set(test_df[column].unique()) - set(train_df[column].unique())
    non_overlapping = only_in_train.union(only_in_test)
    return non_overlapping

def category2num(input_df, columns: list):
    input_ = input_df[columns].copy()
    for column in columns:
        non_overlapping = get_non_overlapping(column)
        if input_df[column].dtype == np.dtype("O"):
            # dtypeがobjectなら欠損は'missing' クラスにする
            input_[column] = input_df[column].fillna("missing")
            input_[column] = input_[column].map(lambda x: x if x not in non_overlapping else "other")
        else:
            # dtypeがint/floatなら欠損は'-1'とする
            input_[column] = input_df[column].fillna(-1)
            input_[column] = input_[column].map(lambda x: x if x not in non_overlapping else -2)

    return input_

In [19]:
train_LE = category2num(train_df, label_encoding_col)
print('train_LE', train_LE.shape)
test_LE = category2num(test_df, label_encoding_col)
print('test_LE', test_LE.shape)
concatenated = pd.concat([train_LE, test_LE], axis=0).reset_index(drop=True)

for column in label_encoding_col:
    le = preprocessing.LabelEncoder()
    le.fit(concatenated[column])
    train_LE[column] = le.transform(train_LE[column])
    test_LE[column] = le.transform(test_LE[column])

train_LE (1997595, 18)
test_LE (390095, 18)


In [20]:
def create_label_encoding_features(input_df, is_test=False):
    if is_test:
        return test_LE
    else:
        return train_LE

### target encoding

In [21]:
def target_encoding(train, test, enc_col):
    group_train = train.groupby([enc_col]).mean()[['target']].reset_index()
    test_copy = test[[enc_col]].copy()
    test_merge = pd.merge(test_copy, group_train, on=[enc_col], how='left')
    test_merge.set_index(test_copy.index, inplace=True)
    test_merge['target'].fillna(train['target'].mean(), inplace=True)
    enc_name = 'TGE_' + enc_col
    test_merge.rename(columns={'target': enc_name}, inplace=True)
    return test_merge.drop(columns=enc_col)

In [22]:
def create_targetencoding_features(input_df, is_test=False):
    
    if is_test:
        # test用 (全データ使用)
        print('TGE for test')
        tgt_all = pd.DataFrame()
        for enc_col in target_enc_col:
            tmp = target_encoding(train_df, test_df, enc_col)
            tgt_all = pd.concat([tgt_all, tmp], axis=1)
        return tgt_all
    
    else:
        # train用 (oof)
        print('TGE for train (TimeSeriesSplit)')
#         kf = KFold(n_splits=5, shuffle=True, random_state=0)
        kf = TimeSeriesSplit(n_splits=10)

        tgt_all = pd.DataFrame()
        for enc_col in target_enc_col:
            tgt_col = pd.DataFrame()

            for train_index, eval_index in kf.split(train_df):
                kf_train = train_df.iloc[train_index]
                kf_eval = train_df.iloc[eval_index]

                tmp = target_encoding(kf_train, kf_eval, enc_col)
                tgt_col = pd.concat([tgt_col, tmp])

            tgt_all = pd.concat([tgt_all, tgt_col], axis=1)
            print(enc_col)
        
        return tgt_all.sort_index()

### Count Encoding

In [23]:
def create_count_encoding_features(input_df, is_test=False):
    out_df = pd.DataFrame()
    for c in count_enc_col:
        series = whole_df[c]
        vc = series.value_counts(dropna=False)

        _df = pd.DataFrame(input_df[c].map(vc))
        out_df = pd.concat([out_df, _df], axis=1)

    out_df = out_df.add_prefix('CE_')
    return out_df

### One-hot encoding

In [24]:
def create_onehot_encoding_features(input_df, is_test=False):
    out_df = pd.DataFrame()
    for c in onehot_enc_col:
        series = input_df[c]
        cat = pd.Categorical(series, categories=whole_df[c].dropna().unique())
        _df = pd.get_dummies(cat)
        _df.columns = _df.columns.tolist()
        _df = _df.add_prefix(c + '=')
        out_df = pd.concat([out_df, _df], axis=1)

    return out_df.add_prefix('OH_')

### 集約系

In [25]:
def create_aggregate_features(input_df, is_test=False):
    return _run_aggregation(input_df, 'uid')

In [26]:
def _run_aggregation(input_df, agg_column):
    _agg_df = pd.concat([
#         whole_df.groupby(agg_column)['last_login_interval'].agg(['mean']).add_prefix('last_login_int_'),
#         whole_df.groupby(agg_column)['first_login_interval'].agg(['mean']).add_prefix('first_login_int_'),
#         whole_df.groupby(agg_column)['max_login_interval'].agg(['mean']).add_prefix('max_login_int_'),
#         whole_df.groupby(agg_column)['frequency'].agg(['mean']).add_prefix('frequency_'),
#         whole_df.groupby(agg_column)['login_frequency'].agg(['mean']).add_prefix('login_frequency_'),
        
        whole_df.groupby(agg_column)['advertiser_id'].nunique(),
        whole_df.groupby(agg_column)['app_id'].nunique(),
        whole_df.groupby(agg_column)['media_app_id'].nunique(),
        whole_df.groupby(agg_column)['campaign_id'].nunique()
    ], axis=1)

    out_df = pd.merge(input_df[agg_column], _agg_df, on=agg_column, how='left')
    out_df = out_df.drop(columns=agg_column).add_suffix('_by_{}'.format(agg_column))
    return out_df

### 四則演算

In [27]:
def feature_operation(input_df):
#     input_df['first_last_login_interval'] = input_df['first_login_interval'] - input_df['last_login_interval']
#     input_df['login_freq_frequency'] = input_df['login_frequency'] - input_df['frequency']
    return input_df

### 連続変数

In [28]:
def create_continuous_features(input_df, is_test=False):
    return input_df[continuous_col].copy()

### date系

In [29]:
def create_date_features(input_df, is_test=False):
    date_df = pd.DataFrame(pd.to_datetime(input_df['imp_at'], utc=True))
    date_df['imp_at'] = date_df['imp_at'].dt.tz_convert('Asia/Tokyo')
#     date_df['day'] = date_df['imp_at'].dt.day
    date_df['hour'] = date_df['imp_at'].dt.hour
#     date_df['total_minute'] = date_df['imp_at'].dt.hour*60+date_df['imp_at'].dt.minute
    date_df['hour_zone'] = pd.cut(date_df['hour'].values, bins=[-np.inf, 6, 12, 18, np.inf]).codes
    date_df['dayofweek'] = date_df['imp_at'].dt.dayofweek
    date_df.drop(columns=['imp_at'], inplace=True)
    return date_df

In [30]:
def _create_whole_imp_at_feature() -> pd.DataFrame:
    """impression が起こった時刻が dayofweek / hour_zone の意味でどれだけ偏っているか集計した data-frame を作成する"""

    imp_at = pd.to_datetime(whole_df['imp_at']) + timedelta(hours=9) # utc -> asia/tokyo
    out_df = pd.DataFrame()
    out_df['hours'] = imp_at.dt.hour
    out_df['dayofweek'] = imp_at.dt.dayofweek
    out_df['hour_zone'] = pd.cut(out_df['hours'].values, bins=[-np.inf, 6, 12, 18, np.inf]).codes

    def _create_pivot(input_df, c, column='dayofweek', values='hours'):
        _df = pd.pivot_table(data=input_df, index=[c], columns=[column], values=values, aggfunc='count')
        _df = _df.fillna(0)
        _df.columns = [column + '=' + str(x) for x in _df.columns]

        # index ごとに正規化して割合にする
        _df = _df.div(_df.sum(axis=1), axis=0)
        return pd.merge(input_df[c], _df, on=c, how='left').drop(columns=[c]).add_prefix(c + '_')

    # uid での集約
    for c in ['uid']:
        _df = out_df.copy()
        _df[c] = whole_df[c]
        _feat = pd.concat([
            _create_pivot(_df, c),
            _create_pivot(_df, c, column='hour_zone', values='dayofweek')  
        ], axis=1)
        out_df = pd.concat([out_df, _feat], axis=1)

    out_df['request_id'] = whole_df['request_id']
    out_df.drop(columns=['hours', 'dayofweek', 'hour_zone'], inplace=True)
    return out_df

In [31]:
datetime_whole_df = _create_whole_imp_at_feature()

In [32]:
def create_imp_at_agg_features(input_df, is_test=False):
    return pd.merge(input_df['request_id'], datetime_whole_df, on='request_id', how='left').drop(columns=['request_id'])

In [33]:
processors = [
    create_continuous_features,
    create_label_encoding_features,
    create_date_features,
    create_targetencoding_features,
    create_count_encoding_features,
    create_onehot_encoding_features,
    create_aggregate_features,
    create_imp_at_agg_features
]

In [34]:
def to_feature(input_df, is_test=False):
    out_df = pd.DataFrame()
    for func in processors:
        _df = func(input_df, is_test)
#         assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
    return out_df

In [35]:
train_feat_df = to_feature(train_df)
test_feat_df = to_feature(test_df, True)
train_feat_df = feature_operation(train_feat_df)
test_feat_df = feature_operation(test_feat_df)
y = train_df['target']

TGE for train (TimeSeriesSplit)
app_id
media_app_id
campaign_id
TGE for test


In [36]:
assert len(train_feat_df) == len(train_df)
assert len(test_feat_df) == len(test_df)

In [37]:
feature_count = len(train_feat_df.columns)
print(feature_count)

78


In [38]:
train_feat_df.tail()

,first_login_interval,max_login_interval,frequency,login_frequency,last_login_interval,from_click,pos,horizontal_width,horizontal_height,vertical_width,vertical_height,horizontal_converted_width,horizontal_converted_height,vertical_converted_width,vertical_converted_height,duration,file_size,converted_file_size,converted_bitrate,adspot_id,adspot_video_format_id,country_code,game_feed_asset_type_id,item_id,os,video_template_id,uid,category_id,game_feed_id,game_template_id,horizontal_mst_advertiser_video_id,vertical_mst_advertiser_video_id,horizontal_converted_rectangle_type_id,vertical_converted_rectangle_type_id,mst_advertiser_id,mst_advertiser_order_id,mst_user_type_id,hour,hour_zone,dayofweek,TGE_app_id,TGE_media_app_id,TGE_campaign_id,CE_advertiser_id,CE_adnw_id,CE_adspot_id,CE_category_id,CE_game_feed_id,CE_uid,CE_game_template_id,CE_mst_advertiser_order_id,CE_mst_user_type_id,OH_auction_type_id=1.0,OH_auction_type_id=2.0,OH_auction_type_id=4.0,OH_header_bidding=0.0,OH_header_bidding=1.0,OH_is_interstitial=1.0,OH_is_interstitial=0.0,OH_user_type_id=1,OH_user_type_id=2,OH_user_type_id=4,OH_user_type_id=3,advertiser_id_by_uid,app_id_by_uid,media_app_id_by_uid,campaign_id_by_uid,uid_dayofweek=0,uid_dayofweek=1,uid_dayofweek=2,uid_dayofweek=3,uid_dayofweek=4,uid_dayofweek=5,uid_dayofweek=6,uid_hour_zone=0,uid_hour_zone=1,uid_hour_zone=2,uid_hour_zone=3
1997590,32994.0,5487.0,1,6.0,651.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,37,2,0,1,1,1,6,108518,1,1347,49,626,2,0,0,43,33,5,8,1,4,0.017895,0.018655,0.129836,48160,941185,856650,2205632,362,9,1286.0,48160,73640,0,1,0,1,0,1,0,0,1,0,0,1,1,1,2,0.111111,0.0,0.0,0.555556,0.111111,0.0,0.222222,0.111111,0.555556,0.111111,0.222222
1997591,984500.0,116145.0,2,1.0,20034.0,1,0,1920.0,1080.0,NaN,NaN,960.0,540.0,NaN,NaN,6.0,57111321.0,876350.0,1000000.0,47,2,0,2,1,1,0,107523,1,1347,66,271,2,1,0,28,101,3,8,1,4,0.027778,0.027778,0.000000,101821,156196,137281,2205632,759,2,NaN,101821,641045,1,0,0,1,0,1,0,0,1,0,0,1,2,2,1,0.000000,0.0,0.0,0.000000,0.500000,0.5,0.000000,0.000000,1.000000,0.000000,0.000000
1997592,38298.0,17147.0,5,1.0,21108.0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,27,0,0,1,893,1,6,108518,1,1271,47,626,2,0,0,110,35,4,8,1,4,0.000000,0.000000,0.000000,20516,42014,42014,2205632,44,2,74251.0,20516,852146,0,1,0,0,0,0,1,0,1,0,0,2,1,1,2,0.000000,0.0,0.0,0.500000,0.500000,0.0,0.000000,0.000000,0.500000,0.500000,0.000000
1997593,651451.0,96344.0,0,1.0,19060.0,1,1,1280.0,720.0,NaN,NaN,NaN,NaN,NaN,NaN,14.0,17103589.0,0.0,0.0,28,3,0,2,1,0,6,108518,1,1347,66,627,2,0,0,117,129,4,8,1,4,0.022727,0.022727,0.009208,1679,128697,128697,2205632,14,1,NaN,1679,852146,1,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000
1997594,55743.0,42167.0,6,1.0,13575.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,24,0,0,1,1,1,6,108518,1,1274,10,626,2,0,0,110,35,4,8,1,4,0.032364,0.032364,0.000000,20516,13915,1828,2205632,30,2,2882.0,20516,852146,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0.000000,0.5,0.0,0.000000,0.500000,0.0,0.000000,0.000000,0.500000,0.000000,0.500000


In [39]:
train_feat_df.to_feather('../input/train_feat_df_{}.f'.format(No))
test_feat_df.to_feather('../input/test_feat_df_{}.f'.format(No))